In [ ]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import make_pipeline
from scipy.stats import normaltest
# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

%matplotlib
ROOT_DIR = './'
print("Ready!")

In [ ]:
def round_to_hour(dt):
    if dt.minute > 30:#date.replace(minute = 0, hour = 0)
        if dt.hour == 23:
            return dt.replace(minute = 0, hour = 0, second = 0)
        else:
            return dt.replace(minute = 0, hour = dt.hour + 1, second = 0) 
    else:
        return dt.replace(minute = 0, second = 0)
    
def round_dt_to_10min(dt):
    try:
        tmp_min = int(str(int(dt.minute/10)) + '0') if dt.minute/10 > 1 else int(dt.minute)
    except ValueError as e:
        print(e, dt)
        return 'Null'
    if (dt.minute - tmp_min) < 5:
        return dt.replace(minute = tmp_min, second = 0)#если разница меньше 5 минут
    else: 
        try:
            if dt.hour == 23 and tmp_min == 50:
                return dt.replace(day = dt.day + 1 ,minute = 0, second = 0, hour = 0) #для 23:55+ переход на некст день
        except ValueError:
            try:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = dt.month + 1) #для 23:55+ переход на некст месяц
            except ValueError:
                return dt.replace(day = 1 ,minute = 0, second = 0, hour = 0, month = 1, year = dt.year + 1)
        if tmp_min == 50:
            return dt.replace(minute = 0, second = 0, hour = dt.hour + 1)#для НН:55+ добавляем час
        return dt.replace(minute = tmp_min + 10, second = 0)
    
def round_to_half_hour(dt):
    #q = int(dt.minute/15)
    #print(q, dt)
    td = timedelta(hours = 1)
    if dt.minute <= 15:
        return dt.replace(minute = 0)
    if dt.minute > 15 & dt.minute < 45:
        return dt.replace(minute = 30)
    if dt.minute >= 45:
        return dt.replace(minute = 0) + td


In [ ]:

def prepare_mes_df(mes_df):
    mes_df = mes_df.replace('Null', np.nan)
    mes_df['34апп.сумма.3нитка'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)
    mes_df['34апп.сумма.4нитка'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)
    mes_df['34апп.сумма.5нитка'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)

    #подача сухого полимера
    mes_df['DK14_SKD_Deg_003.F1'] /=  10
    mes_df['DK14_SKD_Deg_004.F1'] /= 10
    mes_df['DK14_SKD_Deg_005.F1'] /= 10

    mes_df['Суммарная подача пара на 3 нитку, т/час'] = mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_3'].astype(float)
    mes_df['Суммарная подача пара на 4 нитку, т/час'] = mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_3'].astype(float)
    mes_df['Суммарная подача пара на 5 нитку, т/час'] = mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_3'].astype(float)

    formula = 'пар/полимер, ед.изм = ГКал\т'
    mes_df['34апп.удел.сумма.3нитка'] = ((mes_df['DK14_SKD_Deg_003.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_003.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_003.FRC3009'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['34апп.удел.сумма.4нитка'] = ((mes_df['DK14_SKD_Deg_004.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_004.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_004.FR3012'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['34апп.удел.сумма.5нитка'] = ((mes_df['DK14_SKD_Deg_005.F3_1'].astype(float) + mes_df['DK14_SKD_Deg_005.F3_2'].astype(float) + mes_df['DK14_SKD_Deg_005.FR3015'].astype(float)))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
    mes_df['удел.сумм.3нитка'] = (mes_df['Суммарная подача пара на 3 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['удел.сумм.4нитка'] = (mes_df['Суммарная подача пара на 4 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['удел.сумм.5нитка'] = (mes_df['Суммарная подача пара на 5 нитку, т/час'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)
    mes_df['120апп.удел.3нитка'] = (mes_df['DK14_SKD_Deg_003.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_003.F1'].astype(float)
    mes_df['120апп.удел.4нитка'] = (mes_df['DK14_SKD_Deg_004.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_004.F1'].astype(float)
    mes_df['120апп.удел.5нитка'] = (mes_df['DK14_SKD_Deg_005.F3_3'].astype(float))*0.67/mes_df['DK14_SKD_Deg_005.F1'].astype(float)

    
    #Сдвиг во времени от дегазаторов до места взячи анализа
    mes_df['Дата и время'] = mes_df['Дата и время'].astype('datetime64[ns]') - timedelta(minutes = 90)
    
    
    return mes_df

def prepare_lims_df(lims_df):
    try:
        lims_df = lims_df.drop(columns = ['Unnamed: 3','Unnamed: 4','Массовая доля антиоксиданта Агидол-2, Проценты относител.','Unnamed: 6','Unnamed: 8','Unnamed: 9','Unnamed: 11','Unnamed: 12', 'Unnamed: 13','Примечание','Место отбора', "Номер партии"]).rename(columns = {'Дата и время отбора':'Дата и время'}).iloc[:,:3]
    except KeyError:
        lims_df = lims_df.drop(columns = ['Unnamed: 3','Unnamed: 4','Unnamed: 6','Unnamed: 8','Unnamed: 9','Примечание','Место отбора', "Номер партии"]).rename(columns = {'Дата и время отбора':'Дата и время'}).iloc[:,:3]
    lims_df['Массовая доля тримеров, %'] = (lims_df['Массовая доля тримеров, %']/1000).astype(float)
    lims_df['Массовая доля углеводородов, %'] = (lims_df['Массовая доля углеводородов, %']/100).astype(float)
    lims_df['Дата и время'] = lims_df['Дата и время'].astype('datetime64[ns]').apply(round_to_half_hour)
    
    #tdelta = timedelta(minutes = 30)
    #lims_df['Дата и время'] = lims_df['Дата и время'] + tdelta
    return lims_df

def make_final_df(mes_df, lims_df):
    fin_df = mes_df.merge(lims_df, how = 'left', on = 'Дата и время')
    #

    try:
        fin_df.to_excel('fin_df.xlsx')
    except pd.FileCreateError as fce:
        print(fce)
    return fin_df


In [ ]:
lims_data = pd.read_excel(os.path.join(ROOT_DIR, 'Сводный журнал результатов анализов за период ЦЗЛ ПП ПБК.xlsx'),sheet_name = 'Лист1',thousands = ',')
mes_data = pd.read_excel(os.path.join(ROOT_DIR, 'скд.xlsx'),sheet_name = 'скд').drop(columns = [ 'wwRetrievalMode','Дата и время.1'])

addition_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\Лист Microsoft Excel.xlsx",sheet_name = 'Лист1').replace('Null', np.nan).drop(columns = ['wwRetrievalMode'])
addition_data['Дата и время'] = addition_data['Дата и время'].astype('datetime64[ns]') - timedelta(minutes = 90)

mes_df1 = prepare_mes_df(mes_data)
lims_df1 = prepare_lims_df(lims_data)
mes_df1 = mes_df1.merge(addition_data, how = 'left', on = 'Дата и время')
final_df1 = make_final_df(mes_df1, lims_df1)


lims_new_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\Сводный журнал результатов c 18 до 19,5.xlsx",sheet_name = 'Лист2',thousands = ',')
mes_new_data = pd.read_excel("C:\\Users\\bobca\\WorkData\\SKD_DEG\\skd deg new data\\skd18-195_new.xlsx",sheet_name = 'skd18-195_new')
mes_df2 = prepare_mes_df(mes_new_data)
lims_df2 = prepare_lims_df(lims_new_data)
final_df2 = make_final_df(mes_df2, lims_df2)

fin_df = final_df1.append(final_df2)
fin_df.sort_values(by='Дата и время')
fin_df

In [ ]:
#Для drop'a нитки
nit3 = ['DK14_SKD_Deg_003.F1',
 'DK14_SKD_Deg_003.F1_M',
 'DK14_SKD_Deg_003.F2_1',
 'DK14_SKD_Deg_003.F2_2',
 'DK14_SKD_Deg_003.F3_1',
 'DK14_SKD_Deg_003.F3_1.IsOverheat',
 'DK14_SKD_Deg_003.F3_1.T_sat',
 'DK14_SKD_Deg_003.F3_2',
 'DK14_SKD_Deg_003.F3_2.IsOverheat',
 'DK14_SKD_Deg_003.F3_2.T_sat',
 'DK14_SKD_Deg_003.F3_3',
 'DK14_SKD_Deg_003.F4',
 'DK14_SKD_Deg_003.F5',
 'DK14_SKD_Deg_003.F6',
 'DK14_SKD_Deg_003.F7_1',
 'DK14_SKD_Deg_003.F7_2',
 'DK14_SKD_Deg_003.FRC3009',
 'DK14_SKD_Deg_003.FRC3009.IsOverheat',
 'DK14_SKD_Deg_003.FRC3009.T_sat',
 'DK14_SKD_Deg_003.I1',
 'DK14_SKD_Deg_003.I2',
 'DK14_SKD_Deg_003.L1',
 'DK14_SKD_Deg_003.L1_1',
 'DK14_SKD_Deg_003.L1_2',
 'DK14_SKD_Deg_003.L2',
 'DK14_SKD_Deg_003.L2_1',
 'DK14_SKD_Deg_003.L2_2',
 'DK14_SKD_Deg_003.P1',
 'DK14_SKD_Deg_003.P2',
 'DK14_SKD_Deg_003.P2_1',
 'DK14_SKD_Deg_003.Q2',
 'DK14_SKD_Deg_003.Q2_2',
 'DK14_SKD_Deg_003.Q3_1_2',
 'DK14_SKD_Deg_003.Q3_2',
 'DK14_SKD_Deg_003.Q3_3',
 'DK14_SKD_Deg_003.Q4',
 'DK14_SKD_Deg_003.Q5',
 'DK14_SKD_Deg_003.Q6',
 'DK14_SKD_Deg_003.State1',
 'DK14_SKD_Deg_003.State2',
 'DK14_SKD_Deg_003.T1',
 'DK14_SKD_Deg_003.T2',
 'DK14_SKD_Deg_003.T3',
 'DK14_SKD_Deg_003.T4',
 'DK14_SKD_Deg_003.TT100',
 'DK14_SKD_Deg_003.TT101',
 '34апп.сумма.3нитка',
 'Суммарная подача пара на 3 нитку, т/час',
 '34апп.удел.сумма.3нитка',
 'удел.сумм.3нитка',
 '120апп.удел.3нитка',
 'DK14_Shop.REZIM_3N']


nit4 = [ 'DK14_SKD_Deg_004.F1',
 'DK14_SKD_Deg_004.F1_M',
 'DK14_SKD_Deg_004.F2_1',
 'DK14_SKD_Deg_004.F2_2',
 'DK14_SKD_Deg_004.F3_1',
 'DK14_SKD_Deg_004.F3_1.IsOverheat',
 'DK14_SKD_Deg_004.F3_1.T_sat',
 'DK14_SKD_Deg_004.F3_2',
 'DK14_SKD_Deg_004.F3_2.IsOverheat',
 'DK14_SKD_Deg_004.F3_2.T_sat',
 'DK14_SKD_Deg_004.F3_3',
 'DK14_SKD_Deg_004.F3_3.IsOverheat',
 'DK14_SKD_Deg_004.F3_3.T_sat',
 'DK14_SKD_Deg_004.F4',
 'DK14_SKD_Deg_004.F5',
 'DK14_SKD_Deg_004.F6',
 'DK14_SKD_Deg_004.F7_1',
 'DK14_SKD_Deg_004.F7_2',
 'DK14_SKD_Deg_004.FR3012',
 'DK14_SKD_Deg_004.FR3012.IsOverheat',
 'DK14_SKD_Deg_004.FR3012.T_sat',
 'DK14_SKD_Deg_004.FRC3012',
 'DK14_SKD_Deg_004.I1',
 'DK14_SKD_Deg_004.I2',
 'DK14_SKD_Deg_004.L1',
 'DK14_SKD_Deg_004.L1_1',
 'DK14_SKD_Deg_004.L1_2',
 'DK14_SKD_Deg_004.L2',
 'DK14_SKD_Deg_004.L2_1',
 'DK14_SKD_Deg_004.L2_2',
 'DK14_SKD_Deg_004.P1',
 'DK14_SKD_Deg_004.P2',
 'DK14_SKD_Deg_004.P2_1',
 'DK14_SKD_Deg_004.Q2',
 'DK14_SKD_Deg_004.Q2_2',
 'DK14_SKD_Deg_004.Q3_1_2',
 'DK14_SKD_Deg_004.Q3_2',
 'DK14_SKD_Deg_004.Q3_3',
 'DK14_SKD_Deg_004.Q4',
 'DK14_SKD_Deg_004.Q5',
 'DK14_SKD_Deg_004.Q6',
 'DK14_SKD_Deg_004.State1',
 'DK14_SKD_Deg_004.State2',
 'DK14_SKD_Deg_004.T1',
 'DK14_SKD_Deg_004.T2',
 'DK14_SKD_Deg_004.T3',
 'DK14_SKD_Deg_004.T4',
 '34апп.сумма.4нитка',
 'Суммарная подача пара на 4 нитку, т/час',
 '34апп.удел.сумма.4нитка',
 'удел.сумм.4нитка',
 '120апп.удел.4нитка']

nit5 = [ 'DK14_SKD_Deg_005.F1',
 'DK14_SKD_Deg_005.F1.8H',
 'DK14_SKD_Deg_005.F1_M',
 'DK14_SKD_Deg_005.F2_1',
 'DK14_SKD_Deg_005.F2_2',
 'DK14_SKD_Deg_005.F3_1',
 'DK14_SKD_Deg_005.F3_1.IsOverheat',
 'DK14_SKD_Deg_005.F3_1.T_sat',
 'DK14_SKD_Deg_005.F3_2',
 'DK14_SKD_Deg_005.F3_2.IsOverheat',
 'DK14_SKD_Deg_005.F3_2.T_sat',
 'DK14_SKD_Deg_005.F3_3',
 'DK14_SKD_Deg_005.F3_3.IsOverheat',
 'DK14_SKD_Deg_005.F3_3.T_sat',
 'DK14_SKD_Deg_005.F4',
 'DK14_SKD_Deg_005.F5',
 'DK14_SKD_Deg_005.F6',
 'DK14_SKD_Deg_005.F7_1',
 'DK14_SKD_Deg_005.F7_2',
 'DK14_SKD_Deg_005.FR3015',
 'DK14_SKD_Deg_005.FR3015.IsOverheat',
 'DK14_SKD_Deg_005.FR3015.T_sat',
 'DK14_SKD_Deg_005.FRC3015',
 'DK14_SKD_Deg_005.I1',
 'DK14_SKD_Deg_005.I2',#Ток мешалки?
 'DK14_SKD_Deg_005.L1',
 'DK14_SKD_Deg_005.L1_1',
 'DK14_SKD_Deg_005.L1_2',
 'DK14_SKD_Deg_005.L2',
 'DK14_SKD_Deg_005.L2_1',
 'DK14_SKD_Deg_005.L2_2',
 'DK14_SKD_Deg_005.P1',
 'DK14_SKD_Deg_005.P2',
 'DK14_SKD_Deg_005.P2_1',
 'DK14_SKD_Deg_005.Q2',
 'DK14_SKD_Deg_005.Q2_2',
 'DK14_SKD_Deg_005.Q3_1_2',
 'DK14_SKD_Deg_005.Q3_2',
 'DK14_SKD_Deg_005.Q3_3',
 'DK14_SKD_Deg_005.Q4',
 'DK14_SKD_Deg_005.Q5',
 'DK14_SKD_Deg_005.Q6',
 'DK14_SKD_Deg_005.State1',
 'DK14_SKD_Deg_005.State2',
 'DK14_SKD_Deg_005.T1',
 'DK14_SKD_Deg_005.T2',
 'DK14_SKD_Deg_005.T3',
 'DK14_SKD_Deg_005.T4',
 'DK14_SKD_Deg_005.TR103_1',
 'DK14_SKD_Deg_005.TR103_2',
 '34апп.сумма.5нитка',
 'Суммарная подача пара на 5 нитку, т/час',
 '34апп.удел.сумма.5нитка',
 'удел.сумм.5нитка',
 '120апп.удел.5нитка']
print(len(nit5), len(nit4), len(nit3))

In [ ]:
#test
list_5 = ['DK14_SKD_Deg_005.L1_1','DK14_SKD_Deg_005.L1_2',
            'DK14_SKD_Deg_005.L2_1','DK14_SKD_Deg_005.L2_2',
            'DK14_SKD_Deg_005.Q2_2',
            'DK14_SKD_Deg_005.Q3_1_2','DK14_SKD_Deg_005.Q3_2',
            'DK14_SKD_Deg_005.Q3_3','DK14_SKD_Deg_005.Q4',
            'DK14_SKD_Deg_005.F1_M','DK14_SKD_Deg_005.F2_2',
            'DK14_SKD_Deg_005.F3_1.IsOverheat','DK14_SKD_Deg_005.F3_2.IsOverheat',
            'DK14_SKD_Deg_005.F4', 'DK14_SKD_Deg_005.F5',
            'DK14_SKD_Deg_005.F6', 'DK14_SKD_Deg_005.FR3015.IsOverheat',
            'DK14_SKD_Deg_005.I1', 'DK14_SKD_Deg_005.Q2',
            'DK14_SKD_Deg_005.Q5', 'DK14_SKD_Deg_005.Q6',
            'DK14_SKD_Deg_005.State1', 'DK14_SKD_Deg_005.State2',
            'DK14_SKD_Deg_005.TR103_1','DK14_SKD_Deg_005.TR103_2',
            'DK14_SKD_Deg_005.F7_1', 'DK14_SKD_Deg_005.F7_2','DK14_SKD_Deg_005.FRC3015', 'DK14_SKD_Deg_005.T3',# 57.9%, 100.0%, 57.9% missing values
            'DK14_SKD_Deg_005.F3_1.T_sat', 'DK14_SKD_Deg_005.F3_2.T_sat', 'DK14_SKD_Deg_005.FR3015.T_sat',# 6.9% missing values
            'DK14_SKD_Deg_005.T1', 'DK14_SKD_Deg_005.T2',
            'DK14_SKD_Deg_005.F3_3.IsOverheat', 'DK14_SKD_Deg_005.F3_3.T_sat'
         ]


list_4 = ['DK14_SKD_Deg_004.L1_1','DK14_SKD_Deg_004.L1_2','DK14_SKD_Deg_004.L2_1','DK14_SKD_Deg_004.L2_2',
          'DK14_SKD_Deg_004.Q2_2','DK14_SKD_Deg_004.Q3_1_2','DK14_SKD_Deg_004.Q3_2','DK14_SKD_Deg_004.Q3_3',
          'DK14_SKD_Deg_004.Q4','DK14_SKD_Deg_004.F7_1','DK14_SKD_Deg_004.F7_2',
          'DK14_SKD_Deg_004.F1_M','DK14_SKD_Deg_004.F2_2','DK14_SKD_Deg_004.F3_1.IsOverheat',
          'DK14_SKD_Deg_004.F3_2.IsOverheat','DK14_SKD_Deg_004.F3_3.IsOverheat','DK14_SKD_Deg_004.F4',
          'DK14_SKD_Deg_004.F5','DK14_SKD_Deg_004.F6','DK14_SKD_Deg_004.FR3012.IsOverheat',
          'DK14_SKD_Deg_004.FRC3012','DK14_SKD_Deg_004.I1','DK14_SKD_Deg_004.Q2',
          'DK14_SKD_Deg_004.Q5','DK14_SKD_Deg_004.Q6','DK14_SKD_Deg_004.State1',
          'DK14_SKD_Deg_004.State2',
          'DK14_SKD_Deg_004.F3_1.T_sat', 'DK14_SKD_Deg_004.F3_2.T_sat', 'DK14_SKD_Deg_004.FR3012.T_sat',#6.9%  missing values
          'DK14_SKD_Deg_004.F3_3.T_sat',
          'DK14_SKD_Deg_004.T1', 'DK14_SKD_Deg_004.T2'
         ]

list_3 = ['DK14_SKD_Deg_003.L1_1','DK14_SKD_Deg_003.L1_2',
          'DK14_SKD_Deg_003.L2_1','DK14_SKD_Deg_003.L2_2',
          'DK14_SKD_Deg_003.Q2_2', 'DK14_SKD_Deg_003.I1',
          'DK14_SKD_Deg_003.Q3_1_2','DK14_SKD_Deg_003.Q3_2',
          'DK14_SKD_Deg_003.Q3_3','DK14_SKD_Deg_003.Q4',
          'DK14_SKD_Deg_003.F7_1','DK14_SKD_Deg_003.F7_2',
          'DK14_Shop.REZIM_3N','DK14_SKD_Deg_003.F1_M',
          'DK14_SKD_Deg_003.F2_2','DK14_SKD_Deg_003.F3_1.IsOverheat',
          'DK14_SKD_Deg_003.F3_2.IsOverheat','DK14_SKD_Deg_003.F4',
          'DK14_SKD_Deg_003.F5','DK14_SKD_Deg_003.F6',
          'DK14_SKD_Deg_003.FRC3009.IsOverheat','DK14_SKD_Deg_003.Q2',
          'DK14_SKD_Deg_003.Q5','DK14_SKD_Deg_003.Q6',
          'DK14_SKD_Deg_003.State1','DK14_SKD_Deg_003.State2',
          'DK14_SKD_Deg_003.TT100','DK14_SKD_Deg_003.TT101',
          'DK14_SKD_Deg_003.T3', 'DK14_SKD_Deg_003.T4']#72.2%, 55.1%  missing values


list_dop = ['DK14_SKD_Prod.QR1502',#27.9% miss
            'DK14_Shop.AR501_DK2', 'DK14_Shop.AR502_DK2', #100% miss
            'VR010104EKSKD_TempH20.LHA', 'VR010104EKSTEAM_NORTH.LHA', 'VR010104EKSTEAM_SOUTH.LHA',#57.8% miss
            
           ]


list_tmp_4_5 = list_4.copy()
list_tmp_4_5.extend(list_5)
list_tmp_4_5.extend(nit3)
list_tmp_4_5.extend(list_dop)
list_tmp_4_5.append('index')
print('list_tmp_4_5', len(list_tmp_4_5))
df_4_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_4_5)
#pandas_profiling.ProfileReport(df = df_4_5, minimal = True).to_file('df_4_5.html')#to_widgets()  

list_tmp_3_5 = list_3.copy()
list_tmp_3_5.extend(list_5)
list_tmp_3_5.extend(nit4)
list_tmp_3_5.extend(list_dop)
list_tmp_3_5.append('index')
print('list_tmp_3_5',len(list_tmp_3_5))
df_3_5 = fin_df[(fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_5)
#pandas_profiling.ProfileReport(df = df_3_5, minimal = True).to_file('df_3_5.html')#to_widgets()  

list_tmp_3_4 = list_4.copy()
list_tmp_3_4.extend(list_3)
list_tmp_3_4.extend(nit5)
list_tmp_3_4.extend(list_dop)
list_tmp_3_4.append('index')
print('list_tmp_3_4',len(list_tmp_3_4))
df_3_4  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['DK14_SKD_Deg_004.T1'] > 50)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_4)
#pandas_profiling.ProfileReport(df = df_3_4, minimal = True).to_file('df_3_4.html')#to_widgets()  

list_tmp_3_4_5 = list_tmp_4_5.copy()
list_tmp_3_4_5.extend(list_3)
print('list_tmp_3_4_5',len(list_tmp_3_4_5))
df_3_4_5  = fin_df[(fin_df['DK14_SKD_Deg_004.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_003.F1'] > 0.1)\
              & (fin_df['DK14_SKD_Deg_005.F1'] > 0.1)\
              & (fin_df['Суммарная подача пара на 4 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.4нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.4нитка'] < 8)\
              & (fin_df['удел.сумм.4нитка'] < 10)
              & (fin_df['Суммарная подача пара на 3 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.3нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.3нитка'] < 8)\
              & (fin_df['удел.сумм.3нитка'] < 10)\
              & (fin_df['DK14_Shop.REZIM_3N'] == 0)\
              & (fin_df['Суммарная подача пара на 5 нитку, т/час'] > 5)\
              & (fin_df['120апп.удел.5нитка'] < 5)\
              & (fin_df['34апп.удел.сумма.5нитка'] < 8)\
              & (fin_df['удел.сумм.5нитка'] < 10)\
              & (fin_df['DK14_SKD_Prod.Mn'] < 80)
              ].reset_index().drop(columns = list_tmp_3_4_5)
#pandas_profiling.ProfileReport(df = df_3_4_5, minimal = True).to_file('df_3_4_5.html')#to_widgets()  
print('df_3_4:',df_3_4.shape)
print('df_3_5:',df_3_5.shape)
print('df_4_5:',df_4_5.shape)
print('df_3_4_5:',df_3_4_5.shape)

In [ ]:
rename_4_5_dict = {'120апп.удел.4нитка':'120апп.удел_x',\
 '120апп.удел.5нитка':'120апп.удел_y',
 '34апп.сумма.4нитка':'34апп.сумма_x',
 '34апп.сумма.5нитка':'34апп.сумма_y',
 '34апп.удел.сумма.4нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.5нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_004.F1': 'F1_x',
 'DK14_SKD_Deg_004.F2_1':'F2_1_x',
 'DK14_SKD_Deg_004.F3_1':'F3_1_x',
 'DK14_SKD_Deg_004.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_004.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_004.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_004.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_004.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_004.FR3012': 'FR30_x',
 'DK14_SKD_Deg_004.FR3012.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_004.I2':'I2_x',
 'DK14_SKD_Deg_004.L1': 'L1_x',
 'DK14_SKD_Deg_004.L2':'L2_x',
 'DK14_SKD_Deg_004.P1':'P1_x',
 'DK14_SKD_Deg_004.P2':'P2_x',
 'DK14_SKD_Deg_004.P2_1':'P2_1_x',
 'DK14_SKD_Deg_004.T3':'T3_x',
 'DK14_SKD_Deg_004.T4':'T4_x',
 'DK14_SKD_Deg_005.F1':'F1_y',
 'DK14_SKD_Deg_005.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_005.F2_1':'F2_1_y',
 'DK14_SKD_Deg_005.F3_1':'F3_1_y',
 'DK14_SKD_Deg_005.F3_2':'F3_2_y',
 'DK14_SKD_Deg_005.F3_3':'F3_3_y',
 'DK14_SKD_Deg_005.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_005.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_005.FR3015':'FR30_y',
 'DK14_SKD_Deg_005.I2':'I2_y',
 'DK14_SKD_Deg_005.L1':'L1_y',
 'DK14_SKD_Deg_005.L2':'L2_y',
 'DK14_SKD_Deg_005.P1':'P1_y',
 'DK14_SKD_Deg_005.P2':'P2_y',
 'DK14_SKD_Deg_005.P2_1':'P2_1_y',
 'DK14_SKD_Deg_005.T4':'T4_y',
 'Суммарная подача пара на 4 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 5 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.4нитка':'удел.сумм_x',
 'удел.сумм.5нитка':'удел.сумм_y'}

rename_3_5_dict = {'120апп.удел.3нитка':'120апп.удел_x',\
 '120апп.удел.5нитка':'120апп.удел_y',
 '34апп.сумма.3нитка':'34апп.сумма_x',
 '34апп.сумма.5нитка':'34апп.сумма_y',
 '34апп.удел.сумма.3нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.5нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_003.F1': 'F1_x',
 'DK14_SKD_Deg_003.F2_1':'F2_1_x',
 'DK14_SKD_Deg_003.F3_1':'F3_1_x',
 'DK14_SKD_Deg_003.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_003.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_003.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_003.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_003.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_003.FRC3009': 'FR30_x',
 'DK14_SKD_Deg_003.FRC3009.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_003.I2':'I2_x',
 'DK14_SKD_Deg_003.L1': 'L1_x',
 'DK14_SKD_Deg_003.L2':'L2_x',
 'DK14_SKD_Deg_003.P1':'P1_x',
 'DK14_SKD_Deg_003.P2':'P2_x',
 'DK14_SKD_Deg_003.P2_1':'P2_1_x',
 'DK14_SKD_Deg_003.T3':'T3_x',
 'DK14_SKD_Deg_003.T4':'T4_x',
 'DK14_SKD_Deg_005.F1':'F1_y',
 'DK14_SKD_Deg_005.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_005.F2_1':'F2_1_y',
 'DK14_SKD_Deg_005.F3_1':'F3_1_y',
 'DK14_SKD_Deg_005.F3_2':'F3_2_y',
 'DK14_SKD_Deg_005.F3_3':'F3_3_y',
 'DK14_SKD_Deg_005.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_005.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_005.FR3015':'FR30_y',
 'DK14_SKD_Deg_005.I2':'I2_y',
 'DK14_SKD_Deg_005.L1':'L1_y',
 'DK14_SKD_Deg_005.L2':'L2_y',
 'DK14_SKD_Deg_005.P1':'P1_y',
 'DK14_SKD_Deg_005.P2':'P2_y',
 'DK14_SKD_Deg_005.P2_1':'P2_1_y',
 'DK14_SKD_Deg_005.T4':'T4_y',
 'Суммарная подача пара на 3 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 5 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.3нитка':'удел.сумм_x',
 'удел.сумм.5нитка':'удел.сумм_y'}

rename_3_4_dict = {'120апп.удел.3нитка':'120апп.удел_x',\
 '120апп.удел.4нитка':'120апп.удел_y',
 '34апп.сумма.3нитка':'34апп.сумма_x',
 '34апп.сумма.4нитка':'34апп.сумма_y',
 '34апп.удел.сумма.3нитка':'34апп.удел.сумма_x',
 '34апп.удел.сумма.4нитка':'34апп.удел.сумма_y',
 'DK14_SKD_Deg_003.F1': 'F1_x',
 'DK14_SKD_Deg_003.F2_1':'F2_1_x',
 'DK14_SKD_Deg_003.F3_1':'F3_1_x',
 'DK14_SKD_Deg_003.F3_1.T_sat': 'F3_1.T_sat_x',
 'DK14_SKD_Deg_003.F3_2': 'F3_2_x',
 'DK14_SKD_Deg_003.F3_2.T_sat': 'F3_2.T_sat_x',
 'DK14_SKD_Deg_003.F3_3': 'F3_3_x',
 'DK14_SKD_Deg_003.F3_3.T_sat': 'F3_3.T_sat_x',
 'DK14_SKD_Deg_003.FRC3009': 'FR30_x',
 'DK14_SKD_Deg_003.FRC3009.T_sat':'FR30.T_sat_x',
 'DK14_SKD_Deg_003.I2':'I2_x',
 'DK14_SKD_Deg_003.L1': 'L1_x',
 'DK14_SKD_Deg_003.L2':'L2_x',
 'DK14_SKD_Deg_003.P1':'P1_x',
 'DK14_SKD_Deg_003.P2':'P2_x',
 'DK14_SKD_Deg_003.P2_1':'P2_1_x',
 'DK14_SKD_Deg_003.T3':'T3_x',
 'DK14_SKD_Deg_003.T4':'T4_x',
 'DK14_SKD_Deg_004.F1':'F1_y',
 'DK14_SKD_Deg_004.F1.8H':'F1.8H_y',
 'DK14_SKD_Deg_004.F2_1':'F2_1_y',
 'DK14_SKD_Deg_004.F3_1':'F3_1_y',
 'DK14_SKD_Deg_004.F3_2':'F3_2_y',
 'DK14_SKD_Deg_004.F3_3':'F3_3_y',
 'DK14_SKD_Deg_004.F3_3.IsOverheat':'F3_3.IsOverheat_y',
 'DK14_SKD_Deg_004.F3_3.T_sat':'F3_3.T_sat_y',
 'DK14_SKD_Deg_004.FR3012':'FR30_y',
 'DK14_SKD_Deg_004.I2':'I2_y',
 'DK14_SKD_Deg_004.L1':'L1_y',
 'DK14_SKD_Deg_004.L2':'L2_y',
 'DK14_SKD_Deg_004.P1':'P1_y',
 'DK14_SKD_Deg_004.P2':'P2_y',
 'DK14_SKD_Deg_004.P2_1':'P2_1_y',
 'DK14_SKD_Deg_004.T3':'T3_y',
 'DK14_SKD_Deg_004.T4':'T4_y',
 'Суммарная подача пара на 3 нитку, т/час':'Суммарная подача пара_x',
 'Суммарная подача пара на 4 нитку, т/час':'Суммарная подача пара_y',
 'удел.сумм.3нитка':'удел.сумм_x',
 'удел.сумм.4нитка':'удел.сумм_y'}

In [ ]:
#TODO - CHECK T1 T2

In [ ]:
#if p < a:  # null hypothesis: x comes from a normal distribution
a = 0.05

for col in range(dfs[0].shape[1]):
    sts, p = normaltest(tmp_norm[:,col])
    if p > a:
        print('+Sample ' + dfs[0].columns.to_list()[col] + ' looks Gaussian (fail to reject H0)')
    else:
        print('-Sample ' + dfs[0].columns.to_list()[col] + ' does not look Gaussian (reject H0)',p)

In [ ]:
#1136  - всего анализов на органику
#528 от df_4_5
#149 от df_3_4
#184 от 3_5
#Подготовка df (MAIN CELL)
number_of_organic_tests = fin_df[np.isnan(fin_df['Массовая доля углеводородов, %']) == False].shape[0]
print('всего анализов на органику:', number_of_organic_tests)

#df_4_5 = df_4_5.drop(columns = tmp_list)

dfs = []
dfs.append(df_4_5[np.isnan(df_4_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_4[np.isnan(df_3_4['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_5[np.isnan(df_3_5['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))

losing_of_tests = number_of_organic_tests
for i in dfs:
    print(i.shape)
    losing_of_tests -= i.shape[0]
print('Потеряно анализов на органику:', losing_of_tests)

In [ ]:
#TEST CELL append'a линий
number_of_organic_tests = fin_df[np.isnan(fin_df['Массовая доля углеводородов, %']) == False].shape[0]
print('всего анализов на органику:', number_of_organic_tests)

df_4_5_tmp = df_4_5.rename(columns = rename_4_5_dict)
df_3_4_tmp = df_3_4.rename(columns = rename_3_4_dict).drop(columns=['DK14_SKD_Deg_003.T1','DK14_SKD_Deg_003.T2'])
df_3_5_tmp = df_3_5.rename(columns = rename_3_5_dict).drop(columns=['DK14_SKD_Deg_003.T1','DK14_SKD_Deg_003.T2'])

dfs = []
dfs.append(df_4_5_tmp[np.isnan(df_4_5_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_4_tmp[np.isnan(df_3_4_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
dfs.append(df_3_5_tmp[np.isnan(df_3_5_tmp['Массовая доля углеводородов, %']) == False]\
                                                .reset_index()\
                                                .drop(columns = ['Массовая доля тримеров, %', 'Дата и время', 'index'])\
                                                .dropna()\
                                                .astype(float))
sum_df_tmp = dfs[0].append(dfs[1])
sum_df_tmp = sum_df_tmp.append(dfs[2])
dfs.append(sum_df_tmp.dropna(axis = 1))


losing_of_tests = number_of_organic_tests
for i in dfs:
    print(i.shape)
    losing_of_tests -= i.shape[0]
print('Потеряно анализов на органику:', losing_of_tests)
'TEST_tmp'

In [ ]:
dfs[-1].columns

In [ ]:
#Данные для обучения и теста модели
#При объеме данных менее 400 наблюдений
#линейная регрессия не способна обучиться чему-либо
scaler = MinMaxScaler()#StandardScaler()
tmp_df = dfs[-1]

n_arr_norm = scaler.fit_transform(tmp_df)
print('Размер данных=', tmp_df.shape)
#X = tmp_df.drop(columns = ['Массовая доля углеводородов, %'])
#y = tmp_df['Массовая доля углеводородов, %']
X = np.delete(n_arr_norm, tmp_df.columns.to_list().index('Массовая доля углеводородов, %'), 1)
y = n_arr_norm[:,tmp_df.columns.to_list().index('Массовая доля углеводородов, %')]
#X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), random_state=20)#, test_size = 0.2, train_size = 0.8)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20)#, test_size = 0.2, train_size = 0.8)
print('Обучающие датасеты', X_train, y_train)
print('Размер Х_train', len(X_train))
print('Размер Х_test', len(X_test))

In [ ]:
# Коэффициент детерминации R-квадрат для модели с константой принимает значения от 0 до 1.
# Чем ближе значение коэффициента к 1, тем сильнее зависимость.
# При оценке регрессионных моделей это интерпретируется как соответствие модели данным.
# Для приемлемых моделей предполагается, что коэффициент детерминации должен быть хотя бы не меньше 50 %
# (в этом случае коэффициент множественной корреляции превышает по модулю 70 %). 
# Модели с коэффициентом детерминации выше 80 % можно признать достаточно хорошими (коэффициент корреляции превышает 90 %).
# Значение коэффициента детерминации 1 означает функциональную зависимость между переменными.
print('Линейная регрессия, метод наименьших квадратов:')
lr = LinearRegression().fit(X_train, y_train)
print("lr.coef_: {}".format(lr.coef_))
print("lr.intercept_: {}".format(lr.intercept_))
print("R-квадрат на обучающем наборе: {:.2f}".format(lr.score(X_train, y_train)))
print("R-квадрат на тестовом наборе: {:.2f}".format(lr.score(X_test, y_test)))
#lr.predict(X_test)

In [ ]:
#Гребневая регрессия(L2 регуляризация[alpha])
#Оптимальное значение alpha зависит от конкретного используемого набора данных. 
#Увеличение alpha заставляет коэффициенты сжиматься до близких к нулю значений,
#что снижает качество работы модели на обучающем наборе, но может улучшить ее обобщающую способность.
parameters = {'alpha':range(200)}
ridge = Ridge()
ridge_grid = GridSearchCV(ridge, parameters,
cv=5, n_jobs=-1,
verbose=True)
ridge_grid.fit(X_train, y_train)
print('best params',ridge_grid.best_params_)
print('best score',ridge_grid.best_score_)
print('Гребневая регрессия(L2 регуляризация[alpha])')
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 74, 100]:
    ridge = Ridge(alpha = alpha).fit(X_train, y_train)
    print('alpha =', alpha )
    print("R-квадрат на обучающем наборе: {:.2f}".format(ridge.score(X_train, y_train)))
    print("R-квадрат на тестовом наборе: {:.2f}".format(ridge.score(X_test, y_test)))


In [ ]:
#Лассо (L1 регуляризация[alpha])
#Результат L1 регуляризации заключается в том,
#что при использовании лассо некоторые коэффициенты становятся равны точно нулю.
parameters = {'alpha':range(200)}
lasso = Lasso()
lasso_grid = GridSearchCV(lasso, parameters,
cv=5, n_jobs=-1,
verbose=True)
lasso_grid.fit(X_train, y_train)
print('best params',lasso_grid.best_params_)
print('best score',lasso_grid.best_score_)
for alpha in [0, 0.0001, 0.0005,0.001, 0.01, 0.02, 0.005, 0.0075, 0.1, 0.3, 0.5, 1, 3, 10, 15, 50, 100]:
    lasso = Lasso(alpha = alpha).fit(X_train, y_train)
    print("R-квадрат на обучающем наборе: {:.2f}".format(lasso.score(X_train, y_train)))
    print("R-квадрат на контрольном наборе: {:.2f}".format(lasso.score(X_test, y_test)))
    print("Количество использованных признаков: {}".format(np.sum(lasso.coef_ != 0)))
    print('################################################')
#print(lasso.coef_)

In [ ]:
parameters = {'max_depth' : range(20)}
tree = DecisionTreeRegressor(random_state = 0)
tree_grid = GridSearchCV(tree, parameters,
cv=5, n_jobs=-1,
verbose=True)
tree_grid.fit(X_train, y_train)
print('best params',tree_grid.best_params_)
print('best score',tree_grid.best_score_)
tree = DecisionTreeRegressor(random_state=0, max_depth = 4)
tree.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree.score(X_test, y_test)))
print("Важности признаков:\n{}".format(tree.feature_importances_))

In [ ]:
#Случайный лес
# max_features = 10
# Правильность на обучающем наборе: 0.935
# Правильность на тестовом наборе: 0.507

# parameters = {'max_depth' : range(5), 'n_estimators': range(100,150),'max_features':range(10,31)}
# forest = RandomForestRegressor(random_state = 0)
# forest_grid = GridSearchCV(forest, parameters,
# cv=5, n_jobs=-1,
# verbose=True)
# forest_grid.fit(X_train, y_train)
# print('best params',forest_grid.best_params_)
# print('best score',forest_grid.best_score_)
forest = RandomForestRegressor(n_estimators=200, random_state=44)#,max_features = 31)
forest.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(forest.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(forest.score(X_test, y_test)))

In [ ]:
#Градиентный бустинг
for i in [0.0001, 0.0005,0.001, 0.01, 0.02, 0.05, 0.075, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 1]:
    gbrt = GradientBoostingRegressor(random_state=5, learning_rate=i, max_depth=2)
    print('learning_rate=',i)
    gbrt.fit(X_train, y_train)
    print("Правильность на обучающем наборе: {:.3f}".format(gbrt.score(X_train, y_train)))
    print("Правильность на тестовом наборе: {:.3f}".format(gbrt.score(X_test, y_test)))

In [ ]:

# regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
# regr.fit(X_train, y_train)
# Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svr', SVR(epsilon=0.2))])